# LORA Fine-tuning

In [1]:
from datasets import *
import datasets

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np 

### Dataset

In [2]:
# load dataset
dataset = load_dataset('community-datasets/gnad10')

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9245
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1028
    })
})

### Model distilbert

In [4]:
model_checkpoint = "distilbert/distilbert-base-uncased"
model_output = "distilbert-base-uncased-gnad10"

# define label maps
id2label = {0: "Web", 1: "Panorama", 2: "International", 3: "Wirtschaft", 4: "Sport", 5: "Inland", 6: "Etat", 7: "Wissenschaft", 8: "Kultur"}
label2id = {"Web": 0, "Panorama": 1, "International": 2, "Wirtschaft": 3, "Sport": 4, "Inland": 5, "Etat": 6, "Wissenschaft": 7, "Kultur": 8}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=9, id2label=id2label, label2id=label2id, trust_remote_code=True)

# display architecture
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


### preprocess data

In [5]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [6]:
# create tokenize function
def tokenize(batch):

    #tokenize and truncate text
    tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=512)
    print(batch['label'])
    return tokens


# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9245
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1028
    })
})

In [7]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Apply untrained model to text

In [ ]:
# define list of examples
text_list = ["Ex-Chef der Bundespolizei solle insbesondere seine Kontakte in den USA nutzen. Der Volltext dieses auf Agenturmeldungen basierenden Artikels steht aus rechtlichen Gründen nicht mehr zur Verfügung."]
print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
International


### Train model

In [10]:

peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules=["q_lin", "k_lin", "v_lin"])

In [11]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'v_lin', 'k_lin', 'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,039,881 || all params: 68,000,274 || trainable%: 1.5292


In [13]:
# hyperparameters
lr = 1e-3
batch_size = 2
num_epochs = 10

In [ ]:
training_args = TrainingArguments(
    output_dir= model_output,
    learning_rate=lr,
    auto_find_batch_size=True,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# function to show metrics for f1 and accuracy
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {
    'f1': f1,
    'accuracy': accuracy
    }

In [16]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics
)

In [17]:
# train model
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.920700,0.816992,0.727595,0.726654
2,0.758500,0.720075,0.764730,0.763619
3,0.673400,0.726817,0.776137,0.774319
4,0.640600,0.734382,0.791961,0.791829
5,0.578200,0.677302,0.803007,0.803502
6,0.522900,0.629277,0.816532,0.815175
7,0.451400,0.639241,0.824774,0.825875
8,0.375700,0.660590,0.820406,0.821012
9,0.298000,0.723539,0.830811,0.831712
10,0.244900,0.769025,0.836077,0.836576


TrainOutput(global_step=11560, training_loss=0.5572704439757192, metrics={'train_runtime': 4068.5291, 'train_samples_per_second': 22.723, 'train_steps_per_second': 2.841, 'total_flos': 1.25434726714368e+16, 'train_loss': 0.5572704439757192, 'epoch': 10.0})

### Generate prediction

In [18]:
model.to('cpu') # moving to cpu for Windows (can alternatively do 'mps' for Mac)

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
Wirtschaft


### Save lora adapters to HuggingFace

In [ ]:
from huggingface_hub import notebook_login

notebook_login() # ensure token gives write access
hf_name = 'cyrp' # your hf username or org name
model_id = hf_name + "/" + model_output # name the model whatever you want

In [ ]:
model.push_to_hub(model_id) # save model

In [ ]:
trainer.push_to_hub(model_output) # save trainer